In [7]:
!pip install rapidfuzz



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from rapidfuzz import process, fuzz
import unicodedata

# =========================
# 1. Cargar ficheros
# =========================
df_click = pd.read_excel("data_cleaning_clickandboat_final_2026-07-24_26.xlsx")
df_nac = pd.read_excel("turistas_nacional_julio_clean.xlsx")
df_int = pd.read_excel("turistas_internacionales_julio_clean.xlsx")
df_cli = pd.read_csv("aemet_clima_todos-a-abordo.csv")

# =========================
# 2. Definir lista maestra de zonas
# =========================
df_nac["ZONA_PROYECTO"] = df_nac["ZONA_PROYECTO"].astype(str).str.strip()
df_int["ZONA_PROYECTO"] = df_int["ZONA_PROYECTO"].astype(str).str.strip()

zonas_maestras = sorted(df_nac["ZONA_PROYECTO"].dropna().unique().tolist())
print("ZONAS_MAESTRAS:", zonas_maestras)
# Esperado: ['A Coruña', 'Ibiza', 'Málaga']

# =========================
# 3. Función de normalización
# =========================
def normaliza_texto(s):
    if pd.isna(s):
        return ""
    s = str(s).strip().lower()
    # quitar acentos
    s = "".join(
        c for c in unicodedata.normalize("NFD", s)
        if unicodedata.category(c) != "Mn"
    )
    return s

# Pre-normalizamos las zonas maestras para comparar en el mismo espacio
zonas_maestras_norm = [normaliza_texto(z) for z in zonas_maestras]

# Mapa inverso normalizado -> original
map_norm_to_original = dict(zip(zonas_maestras_norm, zonas_maestras))

# =========================
# 4. Mapeos manuales especiales (antes de RapidFuzz)
# =========================
# Clave: textos normalizados, valor: nombre de zona maestro
overrides = {
    "eivissa": "Ibiza"  # caso especial en clima AEMET
}

# =========================
# 5. Función de matching con RapidFuzz
# =========================
def match_zona(valor, choices_norm, norm_to_original, min_score=60):
    """
    valor: texto a emparejar (ciudad/provincia)
    choices_norm: lista de nombres canónicos normalizados
    norm_to_original: dict normalizado -> original
    min_score: umbral mínimo de similitud
    """
    if pd.isna(valor):
        return None

    val_norm = normaliza_texto(valor)
    if not val_norm:
        return None

    # 1) Primero mirar si tenemos override manual
    if val_norm in overrides:
        return overrides[val_norm]

    # 2) Si no, usamos RapidFuzz
    match_norm, score, _ = process.extractOne(
        val_norm,
        choices_norm,
        scorer=fuzz.WRatio
    )

    if score >= min_score:
        return norm_to_original[match_norm]
    else:
        return None  # o "Otros" si prefieres

# =========================
# 6. Aplicar RapidFuzz a Click&Boat y Clima
# =========================

# Click&Boat: columna 'ciudad' -> ZONA_PROYECTO
df_click["ZONA_PROYECTO"] = df_click["ciudad"].apply(
    lambda x: match_zona(x, zonas_maestras_norm, map_norm_to_original)
)

# Clima AEMET: columna 'provincia' -> ZONA_PROYECTO
df_cli["ZONA_PROYECTO"] = df_cli["provincia"].apply(
    lambda x: match_zona(x, zonas_maestras_norm, map_norm_to_original)
)

# =========================
# 7. Chequeos rápidos
# =========================
print("\n=== Mapeo Click&Boat (ciudad -> ZONA_PROYECTO) ===")
print(df_click[["ciudad", "ZONA_PROYECTO"]].drop_duplicates())

print("\n=== Mapeo Clima (provincia -> ZONA_PROYECTO) ===")
print(df_cli[["provincia", "ZONA_PROYECTO"]].drop_duplicates())

print("\nFilas sin zona asignada en Click&Boat:", df_click["ZONA_PROYECTO"].isna().sum())
print("Filas sin zona asignada en Clima:", df_cli["ZONA_PROYECTO"].isna().sum())

# =========================
# 8. Guardar ficheros listos para SQL
# =========================
df_click.to_excel("clickandboat_con_zona_proyecto.xlsx", index=False)
df_nac.to_excel("turistas_nacional_julio_con_zona_proyecto.xlsx", index=False)
df_int.to_excel("turistas_internacionales_julio_con_zona_proyecto.xlsx", index=False)
df_cli.to_csv("aemet_clima_con_zona_proyecto.csv", index=False)

print("\nFicheros guardados. Usa ZONA_PROYECTO como clave común en SQL.")


ZONAS_MAESTRAS: ['A Coruña', 'Ibiza', 'Málaga']

=== Mapeo Click&Boat (ciudad -> ZONA_PROYECTO) ===
     ciudad ZONA_PROYECTO
0    coruña      A Coruña
39    ibiza         Ibiza
690  malaga        Málaga

=== Mapeo Clima (provincia -> ZONA_PROYECTO) ===
    provincia ZONA_PROYECTO
0      MÁLAGA        Málaga
51    EIVISSA         Ibiza
102  A CORUÑA      A Coruña

Filas sin zona asignada en Click&Boat: 0
Filas sin zona asignada en Clima: 0

Ficheros guardados. Usa ZONA_PROYECTO como clave común en SQL.


In [3]:

import pandas as pd
from sqlalchemy import create_engine

# ============== 1. PARÁMETROS DE CONEXIÓN MYSQL ==============
USER = "root"      # ej. "root"
PWD  = "1234"
HOST = "localhost"
PORT = "3306"
DB   = "todos_a_bordo"        # el schema que ves en Workbench

# Crear engine de conexión a MySQL
engine = create_engine(f"mysql+pymysql://{USER}:{PWD}@{HOST}:{PORT}/{DB}")

# ============== 2. CARGAR LOS FICHEROS NUEVOS ==============
# Asegúrate de que el script se ejecuta en la carpeta donde están estos ficheros
# (o pon la ruta completa)

# aemet_clima_con_zona_proyecto  -> CSV
df_clima = pd.read_csv("aemet_clima_con_zona_proyecto.csv")

# clickandboat_con_zona_proyecto -> Excel
df_barcos = pd.read_excel("clickandboat_con_zona_proyecto.xlsx")

# turistas_internacionales_julio_con_zona_proyecto -> Excel
df_tur_int = pd.read_excel("turistas_internacionales_julio_con_zona_proyecto.xlsx")

# turistas_nacional_julio_con_zona_proyecto -> Excel
df_tur_nac = pd.read_excel("turistas_nacional_julio_con_zona_proyecto.xlsx")

# ============== 3. SUBIRLOS REEMPLAZANDO LAS TABLAS ==============
# OJO: if_exists="replace" borra la tabla y la crea de nuevo con estas columnas

# Reemplaza aemet_clima
df_clima.to_sql(
    name="aemet_clima",     # nombre EXACTO de la tabla en MySQL
    con=engine,
    if_exists="replace",
    index=False
)

# Reemplaza clickandboat_barcos
df_barcos.to_sql(
    name="clickandboat_barcos",
    con=engine,
    if_exists="replace",
    index=False
)

# Reemplaza turistas_internacionales_julio
df_tur_int.to_sql(
    name="turistas_internacionales_julio",
    con=engine,
    if_exists="replace",
    index=False
)

# Reemplaza turistas_nacional_julio
df_tur_nac.to_sql(
    name="turistas_nacional_julio",
    con=engine,
    if_exists="replace",
    index=False
)

print("Tablas reemplazadas correctamente en MySQL.")


Tablas reemplazadas correctamente en MySQL.
